In [1]:

# imports
import os
import sys
import types
import json

# figure size/format
fig_width = 10
fig_height = 5
fig_format = 'retina'
fig_dpi = 96

# matplotlib defaults / format
try:
  import matplotlib.pyplot as plt
  plt.rcParams['figure.figsize'] = (fig_width, fig_height)
  plt.rcParams['figure.dpi'] = fig_dpi
  plt.rcParams['savefig.dpi'] = fig_dpi
  from IPython.display import set_matplotlib_formats
  set_matplotlib_formats(fig_format)
except Exception:
  pass

# plotly use connected mode
try:
  import plotly.io as pio
  pio.renderers.default = "notebook_connected"
except Exception:
  pass

# enable pandas latex repr when targeting pdfs
try:
  import pandas as pd
  if fig_format == 'pdf':
    pd.set_option('display.latex.repr', True)
except Exception:
  pass



# output kernel dependencies
kernel_deps = dict()
for module in list(sys.modules.values()):
  # Some modules play games with sys.modules (e.g. email/__init__.py
  # in the standard library), and occasionally this can cause strange
  # failures in getattr.  Just ignore anything that's not an ordinary
  # module.
  if not isinstance(module, types.ModuleType):
    continue
  path = getattr(module, "__file__", None)
  if not path:
    continue
  if path.endswith(".pyc") or path.endswith(".pyo"):
    path = path[:-1]
  if not os.path.exists(path):
    continue
  kernel_deps[path] = os.stat(path).st_mtime
print(json.dumps(kernel_deps))

# set run_path if requested
if r'':
  os.chdir(r'')

# reset state
%reset

def ojs_define(**kwargs):
  import json
  from IPython.core.display import display, HTML

  # do some minor magic for convenience when handling pandas
  # dataframes
  def convert(v):
    try:
      import pandas as pd
    except ModuleNotFoundError: # don't do the magic when pandas is not available
      return v
    if type(v) == pd.Series:
      v = pd.DataFrame(v)
    if type(v) == pd.DataFrame:
      j = json.loads(v.T.to_json(orient='split'))
      return dict((k,v) for (k,v) in zip(j["index"], j["data"]))
    else:
      return v
  
  v = dict(contents=list(dict(name=key, value=convert(value)) for (key, value) in kwargs.items()))
  display(HTML('<script type="ojs-define">' + json.dumps(v) + '</script>'), metadata=dict(ojs_define = True))
globals()["ojs_define"] = ojs_define


C:\Users\kerry\AppData\Local\Temp\ipykernel_41036\1140549349.py:20: DeprecationWarning: `set_matplotlib_formats` is deprecated since IPython 7.23, directly use `matplotlib_inline.backend_inline.set_matplotlib_formats()`
  set_matplotlib_formats(fig_format)


{"C:\\Users\\kerry\\AppData\\Local\\Programs\\Python\\Python310\\lib\\importlib\\_bootstrap.py": 1665532704.0, "C:\\Users\\kerry\\AppData\\Local\\Programs\\Python\\Python310\\lib\\importlib\\_bootstrap_external.py": 1665532704.0, "C:\\Users\\kerry\\AppData\\Local\\Programs\\Python\\Python310\\lib\\codecs.py": 1665532704.0, "C:\\Users\\kerry\\AppData\\Local\\Programs\\Python\\Python310\\lib\\encodings\\aliases.py": 1665532704.0, "C:\\Users\\kerry\\AppData\\Local\\Programs\\Python\\Python310\\lib\\encodings\\__init__.py": 1665532704.0, "C:\\Users\\kerry\\AppData\\Local\\Programs\\Python\\Python310\\lib\\encodings\\utf_8.py": 1665532704.0, "C:\\Users\\kerry\\AppData\\Local\\Programs\\Python\\Python310\\lib\\encodings\\cp1252.py": 1665532704.0, "C:\\Users\\kerry\\AppData\\Local\\Programs\\Python\\Python310\\lib\\abc.py": 1665532704.0, "C:\\Users\\kerry\\AppData\\Local\\Programs\\Python\\Python310\\lib\\io.py": 1665532704.0, "C:\\Users\\kerry\\AppData\\Local\\Programs\\Python\\Python310\\li

In [2]:
from scipy.optimize import fsolve
import numpy as np
from scipy.stats import norm

def callBS(S, K, T, sigma, r, q=0):
    d1 = np.log(S/K) + (r-q+0.5*sigma**2)*T
    d1 /= sigma*np.sqrt(T)
    d2 = d1 - sigma*np.sqrt(T)
    N1 = norm.cdf(d1)
    N2 = norm.cdf(d2)
    return np.exp(-q*T)*S*N1 - np.exp(-r*T)*K*N2

def callIV(C, S, K, T, r, q=0):
  return fsolve(
    lambda x: C - callBS(S, K, T, x, r, q),
    x0=0.3
  )

C = 15
S = 100
K = 100
T = 1
r = 0.05
iv = callIV(C, S, K, T, r)
iv[0]

0.32025830955048284

In [3]:
import yfinance as yf
tick = yf.Ticker('aapl')
price = tick.history()["Close"].iloc[-1].round(2)
rounded = int(5*round(price/5, 0))

In [4]:
price

153.22

In [5]:
import pandas as pd
calls = tick.option_chain("2023-04-21").calls
calls = calls.set_index("strike")
puts = tick.option_chain("2023-04-21").puts
puts = puts.set_index("strike")
calls = calls.loc[rounded-15:rounded+15]
ser1 = calls["impliedVolatility"]
ser2 = puts["impliedVolatility"].loc[rounded-15:rounded+15]
df = pd.concat((ser1, ser2), axis=1)
df.columns = ["Call IV", "Put IV"]
calls

,contractSymbol,lastTradeDate,lastPrice,bid,ask,change,percentChange,volume,openInterest,impliedVolatility,inTheMoney,contractSize,currency
strike,,,,,,,,,,,,,
140.0,AAPL230421C00140000,2023-03-08 16:36:24+00:00,14.82,15.20,15.40,0.799999,5.706129,95,14898,0.334479,True,REGULAR,USD
145.0,AAPL230421C00145000,2023-03-08 16:45:34+00:00,11.20,11.20,11.40,1.000000,9.803922,257,11747,0.310432,True,REGULAR,USD
150.0,AAPL230421C00150000,2023-03-08 16:50:30+00:00,8.00,8.00,8.05,1.050000,15.107916,509,32189,0.297370,True,REGULAR,USD
155.0,AAPL230421C00155000,2023-03-08 16:50:17+00:00,5.15,5.05,5.10,0.850000,19.767439,3199,46486,0.275703,False,REGULAR,USD
160.0,AAPL230421C00160000,2023-03-08 16:50:01+00:00,2.92,2.91,2.93,0.550000,23.206760,1686,44386,0.259529,False,REGULAR,USD
165.0,AAPL230421C00165000,2023-03-08 16:49:14+00:00,1.46,1.44,1.45,0.300000,25.862078,918,33471,0.243538,False,REGULAR,USD
170.0,AAPL230421C00170000,2023-03-08 16:50:01+00:00,0.66,0.67,0.67,0.140000,26.923088,457,43256,0.236092,False,REGULAR,USD


In [6]:
df

,Call IV,Put IV
strike,,
140.0,0.334479,0.280036
145.0,0.310432,0.264045
150.0,0.297370,0.247444
155.0,0.275703,0.232063
160.0,0.259529,0.222359
165.0,0.243538,0.209969
170.0,0.236092,0.160653


In [7]:
tick = yf.Ticker('tsla')
price = tick.history()["Close"].iloc[-1].round(2)
rounded = int(5*round(price/5, 0))
calls = tick.option_chain("2023-04-21").calls
calls = calls.set_index("strike")
puts = tick.option_chain("2023-04-21").puts
puts = puts.set_index("strike")
calls = calls.loc[rounded-15:rounded+15]
ser1 = calls["impliedVolatility"]
ser2 = puts["impliedVolatility"].loc[rounded-15:rounded+15]
df = pd.concat((ser1, ser2), axis=1)
df.columns = ["Call IV", "Put IV"]
df

,Call IV,Put IV
strike,,
165.00,0.694339,0.636722
166.67,0.675723,0.632694
170.00,0.678226,0.618534
175.00,0.668949,0.612858
180.00,0.659671,0.601078
183.33,0.652347,0.604923
185.00,0.651127,0.591313
190.00,0.645389,0.582341
191.67,0.642215,0.581059


In [8]:
from pandas_datareader import DataReader as pdr
vix = pdr('vixcls', 'fred', start=2000).reset_index()
import plotly.graph_objects as go

trace = go.Scatter(
    x = vix.DATE,
    y=vix.vixcls/100,
    hovertemplate = "%{x}<br>%{y:.1%}<extra></extra>"
)
fig = go.Figure(trace)
fig.update_layout(
    xaxis_title="Date",
    yaxis_title="VIX",
    xaxis_title_font = {"size":18},
    yaxis_title_font =  {"size":18},
    template="plotly_white",
    yaxis_tickformat=".0%",
    height=400,  
    width=1000,
    showlegend=False
)
fig.show()
